# How to use SPAMM

The only required inputs to run `SPAMM` are:
1. A source AGN spectrum
2. A list of which components to fit

However, there are numerous optional input arguments available. Since there are dozens of optional parameters, they have been separated out into a configuration file, `parameters.yaml`. An example default file is provided in `spamm/utils/parameters.yaml`. Users will want to modify this file or create their own with the same format. Any parameters file can be specified at runtime- if one is not found, the default will be used.

## Getting Started

`SPAMM` can accept the required input spectrum in three different forms: 
1. `spamm.Spectrum` instance
2. `specutils.Spectrum1D` instance
3. tuple containing the wavelength and flux and optionally, the error on the flux

All three methods will be shown below.

The list of acceptable components to fit are listed below:
- `PL` (Nuclear Continuum)
- `FE` (Iron Forect)
- `HOST` (Host Galaxy)
- `BC` (Balmer Continuum)
- `BPC` (Balmer Pseudo Continuum)
- `CALZETTI_EXT` (Calzetti extinction curves)
- `SMC_EXT` (Small Magellanic Cloud extinction curves)
- `LMC_EXT` (Large Magellanic Cloud extinction curves)
- `MW_EXT` (Milky Way extinction curves)
- `AGN_EXT` (AGN extinction curves)

### From a `spamm.Spectrum` instance

In [ ]:
import numpy as np
import astropy.units as u
from spamm import spamm

In [ ]:
from spamm import Spectrum
from astropy.modeling.powerlaws import PowerLaw1D

wavelength = np.arange(1000, 2000)
# Just generate a simple sample power law for a spectrum
flux = PowerLaw1D(amplitude=1.5e-17, x_0=1500, alpha=-2.0)

# The error on the flux is optional, but let's include it to illustrate how it's done.
flux_error = flux * 0.05

# If no wavelength and flux units are provided upon instantiation, 
# they will be retrieved from the parameters.yaml file.
inspectrum = Spectrum(spectral_axis=wavelength, flux=flux, flux_error=flux_error,
                      spectral_axis_unit=u.AA, flux_unit=u.Unit('erg cm-2 s-1 AA-1'))

# Let's fit only the Nuclear Continuum component.
spamm_out = spamm(complist=["PL"], inspectrum=inspectrum)